<a href="https://colab.research.google.com/github/mrneerajnegi/Data-Science-Projects/blob/auto_ml/Heart%20Attack%20Risk/Heart_Attack_Risk_Prediction_Using_Eval_ML_(Auto_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install evalml

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import evalml
from evalml.automl import AutoMLSearch

In [3]:
df=pd.read_csv("./heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
def applySexvalue(value):
    return "Male" if value==1 else "Female";
df["sex_"]=df["sex"].apply(applySexvalue)

In [5]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output,sex_
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,Male
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1,Male
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1,Female
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1,Male
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1,Female


In [6]:
X=df.drop("output",axis=1)
Y=df["output"]

In [7]:
Xtrain,Xtest,Ytrain,Ytest=evalml.preprocessing.split_data(X,Y,problem_type="binary")

In [8]:
model=AutoMLSearch(Xtrain,Ytrain,problem_type="binary")
model.search()

	High coefficient of variation (cv >= 0.5) within cross validation scores.
	Decision Tree Classifier w/ Label Encoder + Imputer + One Hot Encoder may not perform as estimated on unseen data.


In [9]:
model.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,8,Extra Trees Classifier w/ Label Encoder + Impu...,8,0.421932,0.047606,0.421932,97.312328,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,6,Random Forest Classifier w/ Label Encoder + Im...,6,0.430434,0.039077,0.430434,97.258171,False,"{'Label Encoder': {'positive_label': None}, 'I..."
2,1,Elastic Net Classifier w/ Label Encoder + Impu...,1,0.486867,0.056348,0.486867,96.898700,False,"{'Label Encoder': {'positive_label': None}, 'I..."
3,2,Logistic Regression Classifier w/ Label Encode...,2,0.489937,0.058230,0.489937,96.879144,False,"{'Label Encoder': {'positive_label': None}, 'I..."
4,4,LightGBM Classifier w/ Label Encoder + Imputer...,4,0.516424,0.089586,0.516424,96.710425,False,"{'Label Encoder': {'positive_label': None}, 'I..."
5,3,XGBoost Classifier w/ Label Encoder + Imputer ...,3,0.517511,0.104817,0.517511,96.703502,False,"{'Label Encoder': {'positive_label': None}, 'I..."
6,5,CatBoost Classifier w/ Label Encoder + Imputer,5,0.648022,0.004439,0.648022,95.872157,False,"{'Label Encoder': {'positive_label': None}, 'I..."
7,7,Decision Tree Classifier w/ Label Encoder + Im...,7,5.583465,1.229924,5.583465,64.433808,True,"{'Label Encoder': {'positive_label': None}, 'I..."
8,0,Mode Baseline Binary Classification Pipeline,0,15.698798,0.135402,15.698798,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [17]:
model.describe_pipeline(model.rankings.iloc[0]["id"])


***********************************************************************
* Extra Trees Classifier w/ Label Encoder + Imputer + One Hot Encoder *
***********************************************************************

Problem Type: binary
Model Family: Extra Trees

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
3. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : if_binary
	 * handle_unknown : ignore
	 * handle_missing : error
4. Extra Trees Classifier
	 * n_estimators : 100
	 * max_features : auto
	 * max_depth : 6
	 * min_samples_split : 2
	 * min_weight_fraction_leaf : 0.0
	 * n_jobs : -1

Training
Training for binary problems.
Total training time (including CV): 2.5 seconds

Cross Validation
----------------
             Log Loss Binary  MCC Binary  Gini   AUC  Precision  

In [20]:
best_pipeline=model.best_pipeline

In [27]:
best_pipeline.score(Xtest,Ytest,objectives=["f1","auc"])

OrderedDict([('F1', 0.8611111111111112), ('AUC', 0.9307359307359306)])

In [28]:
#to save the model
best_pipeline.save("model.pkl")